In [ ]:
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

Загрузим датасеты для тренировки модели. Удалим колонку с id пользователя. При тренировке и тестировании модели она не нужна.

In [ ]:
X = pd.read_csv("datas/X_train.zip").drop(['user_id'], axis=1)
X.shape

In [ ]:
X.head()

In [ ]:
y = pd.read_csv("datas/y_train.zip")
y.shape

In [ ]:
y.head()

Отделим тренировочную и тестовую выборки для опредления итоговых параметров модели.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Напишем функцию для поиска наилучших параметров модели.

In [ ]:
def rfc_grid_search(X, y):
    """
    Поиск наилучших параметров для RandomForest.
    Возвращает модель с настроенными параметрами, которая обучена
    на части тренировочных данных. 
    """

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    param_grid = {'n_estimators': range(10, 201, 10),
                  'max_depth': range(2, 21),
                  'criterion': ['gini', 'entropy', 'log_loss'],
                  'min_samples_split': range(2, 11),
                  'min_samples_leaf': range(1, 11)}
    
    grid_search = GridSearchCV(RandomForestClassifier(random_state=0),
                               param_grid=param_grid,
                               cv=5,
                               n_jobs=-1)
    
    grid_search.fit(X_train, y_train.values.ravel())

    print(f'Наилучшие параметры модели {grid_search.best_params_}')

    rfc = grid_search.best_estimator_

    score = rfc.score(X_test, y_test.values.ravel())
    pred_proba = rfc.predict_proba(X_test)
    roc_score = roc_auc_score(y_test, pred_proba[:, 1])

    print(f'Точность на тестовой выборке {score:.3f}')
    print(f'ROC_AUC на тестовой выборке {roc_score:.4f}')

    return rfc


In [ ]:
def rfc_final_train_and_test(model, X_train, X_test, y_train, y_test):
    """
    Финальное обучение модели на полном объеме тренировочных данных
    и проверка на тестовых. Сохранение модели.
    """

    model.fit(X_train, y_train.values.ravel())

    score = model.score(X_test, y_test)
    pred_proba = model.predict_proba(X_test)
    roc_score = roc_auc_score(y_test, pred_proba[:, 1])

    print(f'Финальная точность на тестовой выборке {score:.3f}')
    print(f'Финальный ROC_AUC на тестовой выборке {roc_score:.4f}')
    
    with open(f'models/model_roc_auc_{roc_score:.4f}_.pkl', 'wb') as f:
        pickle.dump(model, f)

    print(f'Модель сохранена в файл model_roc_auc_{roc_score:.4f}_.pkl')

In [ ]:
model = rfc_grid_search(X_train, y_train)

In [ ]:
rfc_final_train_and_test(model, X_train, X_test, y_train, y_test)